# Lab 4: 오류 정정(Error Correction)

동적 회로(Dynamic Circuit)는 또한 양자 컴퓨터의 오류 정정을 크게 도울 수 있는 잠재력을 가지고 있습니다. 양자 오류 정정에 익숙하지 않더라도 걱정하지 않아도 됩니다. 우선 고전 데이터를 사용하여 개념을 설명한 후 양자컴퓨터에 적용하도록 하겠습니다. 

물리적 큐비트는 노이즈에 취약하기 때문에 오류 정정은 양자 컴퓨팅에서 중요합니다. 이것은 회로가 실행될 때 항상 작은 확률로 오류가 발생한 다는 것을 의미합니다. 오류 정정은 이 작은 오류 확률을 처리하며, 발생한 오류를 정정함으로서 최종적으로 정확한 결과를 얻을 수 있게 합니다. 

만약 양자 오류 정정에 대해 더 알고 싶거나, 추가 연습이 필요하다면 **[이 주제에 대한 Qiskit Textbook의 이 챕터](https://learn.qiskit.org/course/quantum-hardware/introduction-to-quantum-error-correction-via-the-repetition-code)** 를 살펴보시기 바랍니다. 

먼저, 랩을 시작하기 위해 필요한 모든 것을 준비해 봅시다. 

In [ ]:
# Importing all the parts we will need.
from typing import List, Optional

from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
from qiskit.providers.fake_provider import FakeManilaV2
from qiskit.visualization import plot_histogram


import warnings

warnings.filterwarnings("ignore")

import math

pi = math.pi

## 고전적 오류 정정 (Classical Error Correction)


오류 정정은 언제 처음 만들어졌을까요?

수신 상태가 좋지 않을 때 누군가와 전화 통화를 하는 경우를 생각해봅시다. 분명하게 말해주려는 여러분의 노력에도 불구하고, 때때로 그 메시지는 상대방에게 잘못 전해지거나 왜곡되게 됩니다. 예를 들어, "금요일에 프레드와 점심 먹자"는 메시지가 "금요일에 프레드를 잠시 막자"로 바뀌는 것 처럼 말이죠. 프레드의 금요일은 엉망이 되겠네요. 

아주 간단한 비트와도 같은 메시지를 보내는 순간이라도, 모든 메시지를 보낼때는 항상 작은 에러가 발생할 가능성이 있습니다. 유선으로 1 비트의 데이터를 보낸다 하더라도 아주 작은 가능성으로 이 비트가 뒤집힐 수 있기 때문에 수신자는 `0`이 아닌 `1`을 받게 될 수 있습니다. 통신 케이블이 오래되어 부식된 경우라면 이런 상황은 더 자주 발생합니다. 데이터를 저장할 때도 같은 현상이 발생할 수 있습니다. CD에 흠집이 있다면 어떻게 될까요. 최악의 경우 CPU의 계산에서도 오류가 발생할 수 있습니다.


고전적인 정보 이론에서 오류의 가능성 문제는 이미 잘 알려진 문제입니다.


비트가 원치않게 뒤집힐 확률(AKA 오류 발생)이 10%라고 상상해 봅시다. 이 것은 우리가 보내는 데이터의 각 비트에 대해 수신된 비트가 올바르지 않을 가능성이 10분의 1이라는 것을 의미합니다.  `0` 메시지를 열번 보냈을 때 수신된 메시지는 `0001000000`일 수 있다는 의미입니다. 이미 이로인해 어떤 문제가 발생할지 쉽게 예상할 수 있을 것입니다.


가장 간단한 해결책 중 하나는 반복을 조금 사용하는 것입니다. 데이터 `1`을 `111`로 부호화(encoding)하거나 `0`도 마찬가지로 `000`으로 부호화하는 경우로, 각 데이터 비트는 하나가 아닌 세 개의 비트를 사용하여 부호화됩니다.

어떻게 이게 해결책이 될 수 있을까요?

만약 지금 `000`을 보냈고 오류가 발생했다면, 수신기는 `001`을 수신했을 수 있습니다. 수신기는 000또는 111을  _반드시_ 받게 됨을 알고 있기 때문에, 이경우 000을 수신한 것으로 추측하여 뒤집어진 하나의 비트를 정정할 것입니다. 이것으로 오류가 정정되었습니다! 


하지만 만약 여러개의 에러가 발생했다면 어떻게 될까요? 보낸 메시지는  `011`로 바뀌었고, 수신기는 그것을  `111`로 받아들일 것입니다. 진정한 오류가 발생한 것이지요! 반복을 사용하는 경우, 이런 상황이 발생 하는 것을 줄일 수 있지만, 완벽하게 없애지는 못합니다. 그렇다면 정확히 몇번을 반복해야 에러율을 줄일 수 있는 것일까요? 한번 탐색해 봅시다!

아래의 코드를 실행하여 에러가 어떻게 바뀌는지, 그 확률을 확인해 봅시다:

In [ ]:
# Probability for a 000 message to contain one error:
p1 = 0.10

# Calculating the probability of a message containing 2 or 3 errors:
p3 = 3 * p1 * p1 * (1 - p1) + p1 * p1 * p1

print("Percent chance the 000 message has one error: {}".format(p1 * 100))
print("Percent chance the 000 message has 2 or 3 errors: {:.4f}".format(p3 * 100))


반복을 사용하면 오류로 인해 메시지가 망가질 확률이 2.8%(2비트 이상 뒤집힐 때)로 줄어듭니다. 1비트만 사용하는 경우 발생하는 10% 순간보다 훨씬 낫습니다.하지만, 그저 오차율 10%의 경우에 해당할 뿐입니다. 코드를 수정하여 다양한 오류율을 만들어 낼 수 있습니다. 시도해 보세요. `p1`의 오류 백분율을 변경하여 이것이 2개 이상의 오류가 발생할 확률에 어떤 영향을 미치는지 확인해 봅시다.

반복이 어떻게 도움이 되는지 이해했다면, 위의 메시지를 "부호화(encoding)"하는 간단한 프로그램을 구현할 수 있습니다. 1비트를 3비트로 늘립니다. 그런 다음 또한 메시지를 "해독(decode)"하기 위한 간단한 프로그램을 작성하여 3비트를 다시 1비트로 되돌립니다. , 훌륭한 첫번째 연습문제처럼 보이네요:


### 연습 문제 1

소개의 의미로, 큐비트 0,1,2를 사용하여 3비트의 상태를 큐비트 4로 단일 비트로 만드는 "해독기(decoder)"를 구현해 봅시다. 규칙은 다음과 같습니다: 

- 000->0
- 001->0
- 010->0
- 100->0
- 111->1
- 110->1
- 101->1
- 011->1

_힌트: 그저 비트를 뒤집으면 됩니다, 따라서 위상 게이트는 필요하지 않습니다. 다만  **[basic gates](https://qiskit.org/documentation/apidoc/circuit_library.html#standard-gates)** 면 충분합니다_

In [ ]:
# Creating a simple decoder for the classical case
def create_decoder(qr: QuantumRegister, cr: ClassicalRegister) -> QuantumCircuit:
    # Expect a bit being encoded in the first 3 qubits and decode it into the 4th qubit
    # Make sure values of the first 3 qubit stays the same

    qc = QuantumCircuit(qr, cr)
    q0, q1, q2, q3 = qr
    (c0,) = cr

    ####### your code goes here #######

    return qc

만든 해석기가 1을 부호화(encode) 한 경우에 대해 어떻게 동작하는지 확인 해 봅시다.


오류 정정 회로가 어떻게 동작하는지 확인하기 위해 각기 다른 입력값에 대해 실행(에러로 1을 지닌 경우를 포함)해 봅시다. 

In [ ]:
# We expect a bit being encoded in the first 3 qubits and decode it into the 4th qubit
qr = QuantumRegister(4)
cr = ClassicalRegister(1)

q0, q1, q2, q3 = qr

# To encode a 1. Change them to test the other encodings.
encoder = QuantumCircuit(qr, cr)
encoder.x(q0)
encoder.x(q1)
encoder.x(q2)

decoder = create_decoder(qr, cr)
qc1 = encoder.compose(decoder)

qc1.draw("mpl")

만든 해석기가 잘 동작한다고 생각한다면, 아래의 셀을 실행해 확인해 봅시다:

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4a

grade_ex4a(create_decoder(qr, cr))

# 양자의 경우

## 무엇이 양자의 경우를 복잡하게 만들까요

고전적인 경우에 그러한 오류 정정 코드를 구현하는 것은 비트를 여러 번 반복하여 얻을 수 있기 때문에 비교적 간단한 편입니다.

물론 더 복잡한 다른 코드들도 존재하지만, 결국 대부분의 코드들은 정보를 반복함으로써 어떤 종류의 중복성을 갖는 것으로 구현합니다.

양자의 경우, 다음의 두 가지 주요 이유로 반복이 쉽지 않습니다:

1. 복제 불가 정리(no cloning theorem)로 인해 큐비트의 상태를 복제 할 수 없다.

2. 큐비트를 측정하면 상태가 붕괴되므로, 얽힌 큐비트들을 취급할때 주의해야 함.

이것은 다소 실망스럽게 들렸겠지만, 그럼에도 불구하고 여전히 양자적 오류 정정을 할 수 있고, 오류 수정을 하고 있습니다.

하지만 이를 위해서는 정보를 저장하기 위해 여러개 개의 큐비트가 필요할 뿐만 아니라 (반복을 위와 비슷하게 구현하기 위해) 추가적인 보조 큐비트가 필요하며 이를 *안정기(stabilizers)*라 부릅니다.

핵심 아이디어는, 보조 큐비트들이 정보를 저장하는 큐비트들과 얽히지 않으면서, 측정시 발생할 수 있는 에러에 대한 힌트를 제공하는 것입니다. 

두 개의 큐비트 쌍을 사용할 것이며, 하나는 데이터를 부호화(encoding) 하는 용도로, 다른 하나는 안정기의 용도로 사용할 것입니다.


# 비트 반전(bit-flip) 반복 코드(repetition code)의 구현 

오류 정정을 구현할 수 있도록 필요한 모든것을 준비해 봅시다. 

In [ ]:
# Setup a base quantum circuit for our experiments
encoding = QuantumRegister(3)
stabilizer = QuantumRegister(2)

encoding_q0, encoding_q1, encoding_q2 = encoding
stabilizer_q0, stabilizer_q1 = stabilizer

# Results of the encoding
results = ClassicalRegister(3)

result_b0, result_b1, result_b2 = results

# For measuring the stabilizer
syndrome = ClassicalRegister(2)

syndrome_b0, syndrome_b1 = syndrome

# The actual qubit which is encoded
state = encoding[0]

# The ancillas used for encoding the state
ancillas = encoding[1:]


# Initializing
def initialize_circuit() -> QuantumCircuit:
    return QuantumCircuit(encoding, stabilizer, results, syndrome)

## 큐비트의 초기화 

양자 상태를 오류로 부터 보호하기 위해서, 우선 양자 상태를 준비해 봅시다. 
일반적으로 양자 상태를 다음의 형태로 준비할 수 있습니다. $$|\Psi_0\rangle \rightarrow |\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$$
아래의 회로에서는 물리적 상태를 다음과 같이 준비합니다.
$$|\Psi_1\rangle = |1\rangle$$

In [ ]:
initial_state = initialize_circuit()

initial_state.x(encoding[0])

initial_state.draw(output="mpl")

## 큐비트의 부호화(encoding)

고전적인 경우와 마찬가지로, 초기 큐비트의 상태를 반복을 사용하여 저장할 것입니다. 

따라서 구현하고자 하는 것은 양자 상태 $|\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$를 부호화기(encoder) 유니터리 $U_{en}|\Psi_0\rangle$를 사용해 $ (\alpha |000\rangle + \beta |111\rangle)$로 부호화하는 것입니다.

이 상태는 얽힌 상태이며, 하나의 큐비트를 측정할 때 다른 두 큐비트의 결과도 함께 결정됩니다.

따라서 CX 게이트를 사용하여 최초의 단일 큐비트 상태를 다음과 같이 얽힌 3 큐비트 상태로 만들 것입니다:

In [ ]:
# Encoding using bit flip code
def encode_bit_flip(qc, state, ancillas):
    qc.barrier(state, *ancillas)
    for ancilla in ancillas:
        qc.cx(state, ancilla)
    return qc


# The circuit encoding our qubit
encoding_circuit = encode_bit_flip(initialize_circuit(), state, ancillas)

# The circuit including all parts so far
complete_circuit = initial_state.compose(encoding_circuit)
complete_circuit.draw(output="mpl")

## 해독기(decoder) 회로 준비

원래 상태를 해독하기 위해, 반대로 작동하는 해독기를 구현해야 합니다, 즉, 해독기 유니터리 $U_{de}|\Psi_0\rangle$는 $ (\alpha |000\rangle + \beta |111\rangle)$를   $|\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$에 매핑합니다.

정확히 반대의 기능이기 때문에, 부호화기를 반전(복소공액, Complex Conjugte)하여 구현합니다:

$U_{de} = U_{en}^\dagger$

In [ ]:
# Decoding (doing the reverse)
def decode_bit_flip(qc, state, ancillas):
    qc.barrier(state, *ancillas)
    for ancilla in ancillas:
        qc.cx(state, ancilla)
    return qc


decoding_circuit = decode_bit_flip(initialize_circuit(), state, ancillas)

decoding_circuit.draw(output="mpl")

## 안정기(Stabilizers) 측정  

위에서 $A$를 대상으로 하고 $B$($B$가 이미 중첩되었거나 얽힌 경우)를 소스로 하는 CX를 사용하여 큐비트 $A$를 다른 큐비트 $B$와 얽을 수 있음을 확인했습니다.


그 다음, $B$를 소스(또는 $A$와 완전히 얽히고 연관된 다른 큐비트)로 $A$를 대상으로 또다른 CX 게이트를 적용하여  얽힘을 풀 수 있습니다.

발생한 잠재적 오류를 감지해 내기 위해 안정기를 측정할 것이기 때문에, 상태를 인코딩하는 큐비트와 얽히지 않도록 하는 것이 중요합니다.

이것들을 감안하여, 각 안정기에 짝수의 CX 게이트를 적용해야 함은 명확합니다. 안정기를 유용하게 만들기 위해 측정 결과는 추가적으로, 비트 반전 오류가 발생했는지의 여부와, 3개의 부호화 큐비트들 중 어디에서 오류가 발생했는 지도 알려 줄 수 있어야 합니다. 

### 연습 2

측정하여 단일 비트 반전 오류를 감지하도록, 신드롬(Syndrome) 비트를 계산합니다. 신드롬 비트를 측정하고 안정기 큐비트를 0 상태로 초기화하는 코드를 포함합시다.


이를 구현하는 여러가지 방법 중, 가장 간단하게 부호화할 수 있는 안정기를 만들어 봅시다: 

00 -> 에러가 발생하지 않음

01 -> 에러가 qubit 0에서 발생함 (첫번째 큐비트)

10 -> 에러가 qubit 1에서 발생함 (두번째 큐비트)

11 -> 에러가 qubit 2에서 발생함 (세번째 큐비트)


*힌트: 3개의 부호화 큐비트들은 완벽하게 얽혀있어야 합니다. 만약 그렇지 않다면, 그 중 하나에 비트 반전 오류가 발생한 것입니다.*

In [ ]:
# Add functions such that the classical bits can be used to see which qubit is flipped in the case a single qubit is flipped.
# Use 2 classical bits for it.
# 0 = 00 = no qubit flipped
# 1 = 01 = first qubit (qubit 0) flipped
# 2 = 10 second qubit (qubit 1) flipped
# 3 = 11 = third qubit (qubit 2) flipped
def measure_syndrome_bit(qc, encoding, stabilizer):
    qc.barrier()
    encoding_q0, encoding_q1, encoding_q2 = encoding
    stabilizer_q0, stabilizer_q1 = stabilizer

    ####### your code goes here #######

    ####### don't edit the code below #######
    qc.barrier()
    qc.measure(stabilizer, syndrome)
    with qc.if_test((syndrome_b0, 1)):
        qc.x(stabilizer_q0)
    with qc.if_test((syndrome_b1, 1)):
        qc.x(stabilizer_q1)

    return qc


syndrome_circuit = measure_syndrome_bit(initialize_circuit(), encoding, stabilizer)

complete_circuit = initial_state.compose(encoding_circuit).compose(syndrome_circuit)
complete_circuit.draw("mpl")

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4b

grade_ex4b(complete_circuit)

## 오류 정정

이제 안정기를 만들고 그것들을 측정함으로써 오류 신드롬을 얻을 수 있게 되었습니다. 물론, 오류가 발생했을 때 인지하게 될 뿐아니라 오류를 정정하게 되길 원합니다.

잠재적 오류를 정정하기 위해 이제 동적 회로를 사용하여 신드롬 측정을 사용합니다. 고전적인 경우와 마찬가지로 최대 1개의 오류만 정정할 수 있습니다. 더 많은 오류를 정정하려면 5개 이상의 부호화 큐비트가 있는 더 긴 코드가 필요합니다.

위의 신드롬 측정 방법을 잘 선택했기 때문에 단일 오류의 경우 어떤 큐비트가 뒤집혔는지 정확히 알수 있으므로 이제 오류를 아주 쉽게 정정할 수 있습니다.



### 연습 3
측정된 신드롬의 결과에 따라 에러를 정정해 봅시다.

*힌트: `circuit.x(0).c_if(<condition>)` 를 신드롬에 따라 X-게이트를 적용하는 것에 사용합니다.*

In [ ]:
# Correct the errors, remember how we encoded the errors above!
def apply_correction_bit(qc, encoding, syndrome):
    qc.barrier()
    encoding_q0, encoding_q1, encoding_q2 = encoding

    ####### your code goes here #######

    return qc


correction_circuit = apply_correction_bit(initialize_circuit(), encoding, syndrome)
complete_circuit = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
)
complete_circuit.draw(output="mpl")

현재 누락된 유일한 부분은 부호화 큐비트를 측정하는 것입니다. 초기 상태를 복구하기 위해 측정하기 전에 해독기 회로를 적용합니다.

모든 것이 오류 없이 완벽하게 작동한다면 초기 큐비트만 측정하는 것으로 충분하지만, 항상 모든 큐비트를 측정하여 문제가 발생했는지 확인하는 것은 아니기 때문입니다.


In [ ]:
def apply_final_readout(qc, encoding, results):
    qc.barrier(encoding)
    qc.measure(encoding, results)
    return qc


measuring_circuit = apply_final_readout(initialize_circuit(), encoding, results)
complete_circuit = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)
complete_circuit.draw(output="mpl")

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4c

grade_ex4c(complete_circuit)

## 테스트 준비 완료

이제 올바른 출력을 얻을 수 있는지 테스트할 수 있는 모든 것이 준비되었습니다. 

올바르게 구현되었는지 확인하기 위해 오류 없이 첫 번째 테스트를 수행합니다:

In [ ]:
# We first choose a simulator as our backend without any noise
backend = Aer.get_backend("qasm_simulator")
# We run the simulation and get the counts
counts = backend.run(complete_circuit, shots=1000).result().get_counts()
# And now we plot a histogram to see the possible outcomes and corresponding probabilities
plot_histogram(counts)

올바른 결과가 얻어진 것을 볼 수 있습니다(상태 1에서 초기 큐비트를 초기화했기 때문에 00 001이 측정되어야 합니다).

보시다시피 부호화에 사용된 다른 큐비트는 예상대로 연산 후 0 상태에 있습니다.

이제 회로가 노이즈 없이 잘 작동한다는 것을 확인했으니 약간의 노이즈를 추가해 봅시다!

이를 위해 노이즈 특성을 포함하여 ibm_manila 백엔드를 모사하는 시뮬레이터를 사용합니다.

*노트: 레이아웃이 단순하기 때문에 이 백엔드를 선택합니다. 아래에서 레이아웃에 대해 자세히 살펴보겠습니다.*

In [ ]:
# We choose a simulator for Manila a 5 qubit device, includes errors
backend = FakeManilaV2()
# We run the simulation and get the counts
counts = backend.run(complete_circuit, shots=1000).result().get_counts()
# We plot a histogram to see the possible outcomes and corresponding probabilities
plot_histogram(counts)

일부 잘못된 결과가 있지만 전반적으로 대부분의 결과는 정확합니다. 이는 좋은 결과로, 노이즈가 있어도 코드가 작동할 수 있음을 의미합니다.

여전히 이번 결과는 에러보정이 없는 결과와 비교할 수 없기때문에 작성한 코드가 얼마나 좋은지 알 수 없으므로, 에러 보정의 단계가 없이 얼마나 잘 동작하는지 확인해 봅시다:

In [ ]:
qc3 = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)


# We choose a again FakeManila
backend = FakeManilaV2()
# let the simulation run
counts = backend.run(qc3, shots=1000).result().get_counts()
# and plot our results
plot_histogram(counts)

부호화 큐비트가 생성된 후 사용되지 않기 때문에 결과가 거의 동일하거나 약간 더 나쁘다는 것을 알 수 있습니다.

이러한 큐비트를 계산에 사용할 때 일반적으로 일부 오류가 발생하지만 여기서는 그렇지 않습니다.

반면 오류 정정 부분은 시간이 걸리는 작업으로 구성되어 있기 때문에 오류를 유발할 수 있습니다.

테스트 목적으로, 약간의 제어된 방식으로 오류를 가하는 회로를 만들어 봅시다:

- 우리가 보정하고자 하는 비트 반전 오류를 만들어 냅니다.

- 서로 다른 큐비트에 독립적으로 세개의 부호화 큐비트에 오류를 만들어 냅니다. 

- 오류가 만들어지는 확률을 선택할 수 있습니다. 

- 오류가 만들어지는 확률은 백분율로 선택할 수 있으며 출력은 정해진 백분률로 오류를 만들어 내는 회로입니다. 

### 연습 문제 4
위에 설명된 방식으로 노이즈를 발생시키는 회로를 만들어 봅시다.

In [ ]:
# Add some errors as defined above (only add errors to the encoding qubits)
def make_some_noise(qc, encoding, syndrome, error_percentage):
    encoding_q0, encoding_q1, encoding_q2 = encoding
    syndrome_b0, syndrome_b1 = syndrome

    ####### your code goes here #######

    return qc


# Constructing a circuit with 10% error rate (for each of the encoding qubit)
noise_circuit = make_some_noise(initialize_circuit(), encoding, syndrome, 10)
noise_circuit.draw(output="mpl")

아래 코드를 사용하여 함수를 테스트하여 오류율이 10%인 회로를 만듭니다.

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4d

grade_ex4d(noise_circuit)

이제 10% 확률로 오류를 도입할 때 오류 정정이 얼마나 잘 작동하는지 테스트해봅시다:

In [ ]:
qc4 = (
    initial_state.compose(encoding_circuit)
    .compose(noise_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)


# We choose again FakeManila
backend = FakeManilaV2()
# let the simulation run
counts = backend.run(qc4, shots=1000).result().get_counts()
# and plot our results
plot_histogram(counts)

결과가 아주 좋지는 않지만 001이 가장 높은 확률로 측정됨을 알 수 있습니다.

여러분은 막 첫번째 오류 정정 코드를 성공적으로 만들었을 뿐 아니라 테스트도 완료 했습니다. 

여러분이 구현한 오류 정정 회로는 오직 비트 반전 오류만을 정정할 수 있습니다. 만약 위상 오류를 보정하고 싶다면 더 많은 큐비트와 다른 방식을 선택해야 합니다. 

더 많은 오류 정정에 대해 알고 싶다면, 다음을 살펴 보세요:

https://learn.qiskit.org/course/ch-labs/lab-10-quantum-error-correction

# 간단히 들여다 보기: 오류 정정과 하드웨어 레이아웃

위에서 실제 하드웨어를 모사하는시뮬레이터를 사용해 보았지만, 실제 양자 장치에서 더 복잡한 것을 실행하려면 레이아웃도 고려해야 합니다.

실제 장비에서는, 이상적인 경우와 같이 모든 큐비트가 서로 연결되어 있지 않기 때문에 장치의 레이아웃이 중요하며 고려해야 합니다. 그렇지 않으면 3개의 cx 작업은 많은 스왑 게이트 연산을 추가로 만들어 낼 수 있고, 이 경우 많은 오류가 이곳에서 발생하게 됩니다.  

실제 장비의 큐비트가 다음과 같이 연결되어 있다고 가정해 봅시다.


    0 - 1 - 2 - 3 - 4 


이렇게 물리적으로 연결된 큐비트들을 어떻게 위에서 작성한 오류 정정 회로에 연결 할 수 있을까요?  


우선, 오류 정정 부분은 여러번 반복해야 하기 때문에, 큐비트의 초기 연결(얽힘)이 오류 정정 부분보다 "더 쉽다"고 가정하여, 위의 회로의 오류 정정 부분을 어떻게 이 큐비트들에 매핑할지를 생각해 봅시다. 

아래 코드를 사용하면 각기 다른 레이아웃이 어떻게 다른 회로를 만들어 낼 수 있는지 확인 할 수 있습니다.


In [ ]:
from qiskit.circuit import IfElseOp

# Fill in a better layout to test!!
initial_layout = [0, 1, 2, 3, 4]

# We use Manila as our potential backend, since it has the wanted layout
backend = FakeManilaV2()

# Temporary workaround for fake backends. For real backends this is not required.
backend.target.add_instruction(IfElseOp, name="if_else")

# And now we transpile the circuit for the backend.
qc_transpiled = transpile(complete_circuit, backend, initial_layout=initial_layout)

qc_transpiled.draw()

초기에 3큐비트를 얽히게 하려면 나중에 오류 정정을 위해 다른 연결성이 필요하기 때문에 어떤 레이아웃도 완벽하진 않습니다, 


[0,4,2,1,3]의 레이아웃이 적절할 수 있는 이유는 그 이후로 정정에 필요한 모든 CX에 대해 직접 연결이 되어 있고, 해당 부분은 잠재적으로 여러 번 실행되기 때문입니다.

또한 이 큐비트들로 초기 얽힘 설정을 더 잘 해낼 수 있는 방법에 대해서도 생각해 볼 수 있습니다, 이는 안정기로 사용되는 큐비트들을 우선 회로를 구성 할 수도 있기 때문입니다. 

상당히 다른 결과들을 얻을 수 있음을 확인하기 위해, 다음의 레이아웃에 큐비트를 어떻게 매핑할지 상상해 봅시다. 


    2 - 3 - 4 
            |
           15
            |
           22

위의 레이아웃과 유사하게, [2,4,22,3,15]도 가능하며, 필요한 CX 게이트들이 모두 직접 연결되어 있기 때문입니다.  

두 레이아웃 모두 단순한 일직선 연결이기 때문에 이러한 레이아웃 간에 실제로 차이가 없음을 알 수 있습니다. 하지만 백엔드 IBM_Quito를 보면 다른 레이아웃을 지닌 5큐비트 장치도 있음을 알 수 있습니다. 이 연습에서는 레이아웃이 더 적절하기 때문에 마닐라를 선택해 사용했습니다. 

이 작은 예제는 실제 하드웨어에서 큐비트의 레이아웃에 신경을 써야 함을 보여줍니다! 


**이 것은 보너스 연습에서 특별히 더 중요합니다!**